### load modules and download pre-trained ESM models

In [36]:
import torch
import esm

# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
#model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
#model, alphabet = esm.pretrained.esm2_t48_15B_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

import pandas as pd
import numpy as np

### Initialize seed for reproducibility
import numpy as np
np.random.seed(0)
import math

### Data Wrangling and Plots
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from keras.models import Sequential
from keras.layers import Dense
import keras

2024-09-07 17:23:33.321224: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-07 17:23:33.341691: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### REad mutated data

In [16]:

data1 = pd.read_csv('../data/sdab_data_crd3_mutations.csv', sep = '\t')

data1 = data1[data1.prob_mut>0.5]

In [24]:
data  = []
for i, row in data1.iterrows():
    data.append((row.id, row.mut_seq))

### generate the embeddings

In [25]:
sequence_representations_list = []
chunk_size = 25
for i in range(0, len(data), chunk_size):
    chunk = data[i:i+chunk_size]
    print(i+chunk_size)
    batch_labels, batch_strs, batch_tokens = batch_converter(chunk)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True) # ESM 650M
        #results = model(batch_tokens, repr_layers=[36], return_contacts=True) # ESM 3B
        #results = model(batch_tokens, repr_layers=[48], return_contacts=True) # ESM 15B
    token_representations = results["representations"][6] # ESM 650M
    #token_representations = results["representations"][36] # ESM 3B
    #token_representations = results["representations"][48] # ESM 15B

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    sequence_representations_list.append(sequence_representations)

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500
525
550
575
600
625
650
675
700
725
750
775


In [27]:
flat_list = [item for sublist in sequence_representations_list for item in sublist]

In [28]:
X = torch.stack(flat_list, dim=0).cpu().detach().numpy()
X.shape

(764, 320)

### save into csv files

In [30]:
np.savetxt("/home/bionets-og86asub/Documents/hackathon-bayer/earth-ml-sensitivity/data/mutated_embedding.csv", X, delimiter=",")


In [ ]:
## Retrain model from scratch with all data and 88 epochs (early stopping in previous test)

In [37]:
def create_model():
    model = Sequential([
        keras.layers.Dense(128, 
                           input_shape = (x.shape[1],), # input shape is the number of predictors
                           activation = 'relu'), # Input Layer
        keras.layers.Dense(256, 
                           activation = 'relu'), # Hidden Layer (mean of neurons in the input and output layers.)
        keras.layers.Dense(256, 
                           activation = 'relu'), # Hidden Layer (mean of neurons in the input and output layers.)
        keras.layers.Dense(256, 
                           activation = 'relu'), # Hidden Layer (mean of neurons in the input and output layers.)
        keras.layers.Dense(1, 
                           activation = 'linear') # If the NN is a regressor, then the output layer has a single node.
    ])
    
    model.compile(
        loss = 'mae',
        optimizer = 'adam',
        metrics= ['mae']
    )
    
    model.summary()

    return model

In [38]:
data = pd.read_csv("/home/bionets-og86asub/Documents/hackathon-bayer/sdab_data_master_list_t6.csv", header = None)
data.head()

y = pd.read_excel("/home/bionets-og86asub/Documents/hackathon-bayer/TEMPRO/paper_results/sdab_data.xlsx")
y = y.tm
x = data

In [40]:
model = create_model()
model_history = model.fit(x, y, epochs=88, batch_size=32)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 128)            │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 205,953 (804.50 KB)

 Trainable params: 205,953 (804.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 62.6514 - mae: 62.6514   
Epoch 2/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 13.5145 - mae: 13.5145 
Epoch 3/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.8239 - mae: 9.8239   
Epoch 4/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.2256 - mae: 8.2256 
Epoch 5/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.3250 - mae: 8.3250 
Epoch 6/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.0353 - mae: 8.0353 
Epoch 7/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.5118 - mae: 7.5118 
Epoch 8/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.3713 - mae: 7.3713 
Epoch 9/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.6871 - mae: 6.6871 
Epoch 10/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.8396 - mae: 7.8396 
Epoch 11/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.6590 - mae: 7.6590 
Epoch 12/88
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.9720 - mae: 6.9720 
Epoch 13/88
18/18 ━━━━━━━━━━━

In [43]:
## Predict mutated sequences

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [47]:
predictions = model.predict(X)
data1['predictions'] = predictions

In [49]:
data1.to_csv('../data/mutated_seq_with_predictions.tsv', header = True, sep = '\t')